In [0]:
%run "../00_config/01_urls"

In [0]:
%run "../00_config/00_adls_config"

In [0]:
from pyspark.sql.functions import when, col, regexp_replace, trim
from datetime import datetime
df = spark.read.csv(
    f"{BRONZE_LAYER_PATH}city_data/city.csv",
    header=True,
    inferSchema=True,
)

df_cleaned = df \
    .withColumnRenamed("lat", "latitude") \
    .withColumnRenamed("lon", "longitude") \
    .withColumn("state",when(col("state").isNull(), trim(regexp_replace(col("city"), r"(?i)^thành\s+ph[ốo]\s*", ""))).otherwise(trim(regexp_replace(col("state"),"Province",""))))\
    .select("country","state","latitude","longitude")

current_date_path = datetime.now().strftime("%Y/%m/%d")
silver_output_path=f"{SILVER_LAYER_PATH}city_data_cleaned/dt={current_date_path}/"
df_cleaned.write.format("delta").mode("append").save(silver_output_path)